In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

### Load your data

In [4]:
#loader = UnstructuredPDFLoader("./data/field-guide-to-data-science.pdf")
loader = PyPDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [5]:
data = loader.load()

In [6]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 126 document(s) in your data
There are 0 characters in your document


### Chunk your data up into smaller documents

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [8]:
print (f'Now you have {len(texts)} documents')

Now you have 258 documents


### Create embeddings of your documents to get ready for semantic search

In [10]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/Users/nishank/opt/anaconda3/lib/python3.8/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
OPENAI_API_KEY = '...'
PINECONE_API_KEY = '...'
PINECONE_API_ENV = '...'

In [16]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [17]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain-test"

In [18]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'www-authenticate': 'API key is missing or invalid for the environment "asia-northeast1-gcp". Check that the correct environment is specified.', 'content-length': '121', 'date': 'Mon, 24 Apr 2023 17:23:45 GMT', 'server': 'envoy'})
HTTP response body: API key is missing or invalid for the environment "asia-northeast1-gcp". Check that the correct environment is specified.


In [14]:
query = "What are examples of good data science teams?"
docs = docsearch.similarity_search(query, include_metadata=True)

### Query those docs to get your answer back

In [16]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [17]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [22]:
query = "What is the collect stage of data maturity?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [23]:
chain.run(input_documents=docs, question=query)

' The collect stage of data maturity focuses on collecting internal or external datasets. Examples include gathering sales records and corresponding weather data.'